In [1]:
import cv2
import os
import tensorflow
from tensorflow.keras.preprocessing import image

In [3]:
data = []
categories = ['with_mask', 'without_mask']
for category in categories:
	path = os.path.join('/content/drive/MyDrive/facemask/train', category)
	label = categories.index(category)
	for file in os.listdir(path):
		img_path = os.path.join(path, file)
		img = cv2.imread(img_path)
		img = cv2.resize(img, (224, 224))
		data.append([img, label])


In [4]:
import random
random.shuffle(data)

In [5]:
x = []
y = []
for features, label in data:
    x.append(features)
    y.append(label)

In [6]:
import numpy as np
x = np.array(x)
y = np.array(y)

In [7]:
x = x/255

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [9]:
from tensorflow.keras.applications.vgg16 import VGG16

In [10]:
vgg = VGG16()

553476096/553467096 [==============================] - 5s 0us/step


In [11]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [12]:
from tensorflow.keras import Sequential

In [13]:
model = Sequential()

In [14]:
for layer in vgg.layers[:-1]:
    model.add(layer)

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [16]:
for layer in model.layers:
    layer.trainable = False

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [18]:
from tensorflow.keras.layers import Dense

In [19]:
model.add(Dense(1, activation='sigmoid'))

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [21]:
model.compile(optimizer='Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [22]:
model.fit(x_train, y_train, epochs = 5, validation_data=(x_test, y_test))

Epoch 1/5
21/21 [==============================] - 397s 19s/step - loss: 0.5855 - accuracy: 0.7143 - val_loss: 0.5439 - val_accuracy: 0.6564
Epoch 2/5
21/21 [==============================] - 395s 19s/step - loss: 0.4303 - accuracy: 0.8095 - val_loss: 0.3166 - val_accuracy: 0.9264
Epoch 3/5
21/21 [==============================] - 394s 19s/step - loss: 0.3151 - accuracy: 0.8955 - val_loss: 0.2715 - val_accuracy: 0.9141
Epoch 4/5
21/21 [==============================] - 395s 19s/step - loss: 0.2468 - accuracy: 0.9370 - val_loss: 0.2252 - val_accuracy: 0.9448
Epoch 5/5
21/21 [==============================] - 398s 19s/step - loss: 0.2126 - accuracy: 0.9478 - val_loss: 0.2020 - val_accuracy: 0.9387


In [84]:
model.save("model.h5")